In [ ]:
import sys
import os

# Get parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add parent directory to sys.path
sys.path.append(parent_dir)

In [1]:
from pydantic import BaseModel
from typing import List, Dict, Tuple, Self, Optional
from bs4 import BeautifulSoup
import re
import feedparser
from tqdm import tqdm
import requests
import time
from deals import ScrapedDeal

# automatically reload changed modules
%load_ext autoreload
%autoreload 2

## Testing RSS feed

In [5]:
# feeds = []

# RSS feed URL
rss_url = "https://www.admcars.com/rssfeed.php"

# Parse the feed
feed = feedparser.parse(rss_url)

# Print feed title
print("Feed Title:", feed.feed.title)

Feed Title: American Dream Machines RSS 2.0 Feed


In [6]:
print(feed.entries[1].keys())
print(feed.entries[1].title)
print(feed.entries[1].summary)
print(feed.entries[1].link)
print(type(feed.entries[1]))

dict_keys(['title', 'title_detail', 'summary', 'summary_detail', 'links', 'link', 'id', 'guidislink', 'published', 'published_parsed'])
1958 Buick Century Extensive Restore, 364 Auto PS PT
1958 Buick Century Extensive Restore, 364 Auto PS PT<br />
			Stock # 5082, Mileage: 0, VIN # 6E2002708<br />
			Price: $94,900<br />
			Exterior Color: Black, Interior Color: Red<br />
			<h3><strong>Elegant Black 1958 Buick Century convertible with extravagant show chrome and shimmering diamond grill. &nbsp;This impressive piece of automotive art was owned by one family for over 50 years and handed down from father to son and has undergone an extensive restoration with attention to detail, with massive show chrome all the way around. One of only 2588 built in 1958 and is showing only 59 miles since completion. The new luxurious red interior was restored to a very impressive level with new carpet, seat cover, door panels and trim and shows like new. The interior is optioned with power windows and po

## Combine relevant parts of description into a summary for pricing model

In [6]:
soup = BeautifulSoup(feed.entries[1].summary, 'html.parser')
soup.find('h3').find('strong').get_text(separator='\n', strip=True)

'Elegant Black 1958 Buick Century convertible with extravagant show chrome and shimmering diamond grill. \xa0This impressive piece of automotive art was owned by one family for over 50 years and handed down from father to son and has undergone an extensive restoration with attention to detail, with massive show chrome all the way around. One of only 2588 built in 1958 and is showing only 59 miles since completion. The new luxurious red interior was restored to a very impressive level with new carpet, seat cover, door panels and trim and shows like new. The interior is optioned with power windows and power seats. \xa0The dash was also restore to a high level, repainted with a new dash pad and refurbished instrument cluster showing just 59 miles. \xa0Under the hood is factory correct and just as impressive showcasing its 364 cubic inch Fireball nailhead designed engine, boasting 300 horsepower and 400 foot pounds of torque with 10-1 compression, topped off with a Quadrajet 4-barrel carbu

In [7]:
for line in soup.find_all('li'):
    print(line.get_text(separator='\n'))

An extensive restoration was preformed on this black Buick Century Convertible  
Professionally painted in Black with white convertible top
Restored Fashionaire Dynastar diamond grille
Re-chromed front and rear bumpers and tail trim 
Jeweled in beautiful chrome and stainless all the way around
New Century side trim 
Newer white power top
Full size wheel covers
Wide whitewall tires
Opulent restored red interior
Impeccable restored dash with new dash pad
Refurbished instrument cluster with odometer showing just 59 miles 
Power windows and power seats 
New seat covers, door panels and carpet 
Restored steering wheel and new chrome horn ring and horn 
In dash clock 
Pushbutton Sonomatic radio
Show quality and factory correct engine compartment 
364 cubic inch Fireball Nailhead V8 engine
Flight Pitch Dynaflow automatic transmission
Power steering
Hydraulic brakes with aluminum drums on the front 
Structurally beautiful metal structure 
Very solid original floors and frame 
Ball joint front 

In [29]:
def get_price(text: str) -> int:
    """Finds price from a text string."""
    # Find the line containing "Price:"
    for line in text.split("\n"):
        if "Price:" in line:
            raw_price = line.split("Price:")[-1].strip()  # Extract price value
            clean_price = re.sub(r"[^\d]", "", raw_price)  # Removes all non-digit characters
            if clean_price:
                clean_price = int(clean_price)
                return clean_price
            else:
                return None

In [10]:
feed.entries[1]

{'title': '1958 Buick Century Extensive Restore, 364 Auto PS PT',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://www.admcars.com/rssfeed.php',
  'value': '1958 Buick Century Extensive Restore, 364 Auto PS PT'},
 'summary': '1958 Buick Century Extensive Restore, 364 Auto PS PT<br />\n\t\t\tStock # 5082, Mileage: 0, VIN # 6E2002708<br />\n\t\t\tPrice: $94,900<br />\n\t\t\tExterior Color: Black, Interior Color: Red<br />\n\t\t\t<h3><strong>Elegant Black 1958 Buick Century convertible with extravagant show chrome and shimmering diamond grill. &nbsp;This impressive piece of automotive art was owned by one family for over 50 years and handed down from father to son and has undergone an extensive restoration with attention to detail, with massive show chrome all the way around. One of only 2588 built in 1958 and is showing only 59 miles since completion. The new luxurious red interior was restored to a very impressive level with new carpet, seat cover, door pan

In [11]:
# price function works in both cases (has price / doesn't have price)
print(price(feed.entries[0].summary))
print(price(feed.entries[1].summary))

None
94900


In [42]:
def get_price_and_description(deal: feedparser.util.FeedParserDict) -> Tuple[str, int]:
    """Combines the text from <h3><strong> and all <li> elements into a single string.
       Extracts price. 
    """
    soup = BeautifulSoup(deal.summary, 'html.parser')
    # extract all text
    text = soup.get_text(separator="\n")
    # get price
    price = get_price(text)
    if not price:
        return None
    # body of description is in h3 header
    h3_element = soup.find('h3')
    if h3_element:
        strong_element = h3_element.find('strong')
        if strong_element:
            h3_text = strong_element.get_text(separator='\n', strip=True)
        else:
            h3_text = ''
    else:
        h3_text = ''
    # Initialize the combined string with the h3_element text
    combined_string = h3_text + "\n" # Add a newline to seperate the h3 text and list items.

    # list elements contain more detailed info on repairs, build, etc.
    for line in soup.find_all('li'):
        combined_string += line.get_text(separator='\n', strip=True) + '\n'
        
    return combined_string.rstrip('\n'), price # remove last new line from combined string

In [37]:
from config import CARS

def generate_year_pattern(start_year, end_year) -> str:
    """
    Generates a regex pattern to match any year between start_year and end_year (inclusive).
    """
    return rf"\b({'|'.join(str(year) for year in range(start_year, end_year + 1))})\b"

def contains_year_and_model(text: str, model: str, start_year: int, end_year: int) -> Tuple[Optional[str], Optional[str]]:
    """
    Extracts values for year and model from title text if exists.
    """
    # Regex pattern for a 4-digit year within range
    year_pattern = generate_year_pattern(start_year, end_year)
    
    # Check if model and year exist
    year_match = re.search(year_pattern, text)
    model_match = re.search(rf"\b{model}\b", text, re.IGNORECASE)  # Case-insensitive match
    
    found_year = year_match.group(0) if year_match else None
    found_model = model_match.group(0) if model_match else None 
    
    return found_year, found_model
# get year and model from feed
# if year and model in CARS
# add to list

def match_model_and_year(title: str) -> Tuple[Optional[str], Optional[str], Optional[str]]:
    make = None
    model = None
    year = None
    
    for car in CARS:
        start_year = car['YearFrom']
        end_year = car['YearTo']
        car_model = car['Model']
        car_make = car['Make']
        make = car_make
        
        year, model = contains_year_and_model(title, car_model, start_year, end_year)    
    
        if model:
            make = car_make
            break
              
    return make, model, year


In [17]:
generate_year_pattern(1992, 1997)

'\\b(1992|1993|1994|1995|1996|1997)\\b'

In [20]:
for car in CARS:
    print(car['YearFrom'])
    print(car['YearTo'])
    print(car['Model'])
    print(car['Make'])

1965
1971
Mustang
Ford
1968
1970
Charger
Dodge
1966
1977
Bronco
Ford
1967
1969
Camaro
Chevrolet


In [39]:
match_model_and_year(feed.entries[13].title)

('Chevrolet', 'Camaro', '1967')

In [40]:
get_price_and_description(feed.entries[13])

('Striking Electric Blue, 1967 Rally Sport, Pro-Tour Chevrolet Camaro Convertible with RS hidden headlights and an extensive list of custom modifications, making it a one of a kind muscle machine that was built for show and go. Under the hood is a show stopping, chromed out, engine compartment, showcasing a \xa0fuel injected 5.7L LT1 V8 engine, accompanied by a chrome serpentine belt system, nicely optioned with newer chrome AC, chrome power steering, \xa04-wheel power disc brakes and with a 4-speed 700R4 automatic overdrive transmission to kick this radical Camaro into high gear. \xa0Fully custom two tone gray bucket seat interior with dark gray carbon fiber pattern seat inserts and lots of upgrades; like power windows, Vintage air conditioning, brand new black console with new working Camaro gauges, tach, tilt steering and leather wrapped steering wheel, topped off with a newer\xa0black canvas convertible top that fits snug. This Electric Blue Camaro sounds even better than it looks,

In [43]:
deals = []
for entry in tqdm(feed.entries):
    title = entry.title
    make, model, year = match_model_and_year(title)
    if model and year:
        result = get_price_and_description(entry)
        if isinstance(result, tuple) and len(result) == 2:
            description, price = result
            deals.append({
                'title': title,
                'make': make,
                'model': model,
                'year': year,
                'price': price,
                'description': description
            })


100%|██████████| 90/90 [00:00<00:00, 958.95it/s]


In [44]:
deals

[{'title': '1967 Chevrolet Camaro Electric Blue 5.7L 700R4 AC PS PB',
  'make': 'Chevrolet',
  'model': 'Camaro',
  'year': '1967',
  'price': 79900,
  'description': 'Striking Electric Blue, 1967 Rally Sport, Pro-Tour Chevrolet Camaro Convertible with RS hidden headlights and an extensive list of custom modifications, making it a one of a kind muscle machine that was built for show and go. Under the hood is a show stopping, chromed out, engine compartment, showcasing a \xa0fuel injected 5.7L LT1 V8 engine, accompanied by a chrome serpentine belt system, nicely optioned with newer chrome AC, chrome power steering, \xa04-wheel power disc brakes and with a 4-speed 700R4 automatic overdrive transmission to kick this radical Camaro into high gear. \xa0Fully custom two tone gray bucket seat interior with dark gray carbon fiber pattern seat inserts and lots of upgrades; like power windows, Vintage air conditioning, brand new black console with new working Camaro gauges, tach, tilt steering a

In [26]:
for i, entry in enumerate(feed.entries[:20]):
    print(f"{i}:{entry.title}")

0:1932 Auburn 898-A Exquisite Black &Silver Boattail Speedster Red Int. Extensive Restoration
1:1958 Buick Century Extensive Restore, 364 Auto PS PT
2:1959 Cadillac Fleetwood Series 75 Limousine, 26603 Miles
3:1960 Cadillac Fleetwood Extravgant Black Series 75 Pro-Tour Limousine Tan Napa Leather 390 Auto AC
4:1953 Chevrolet 3100 Brandywine 5 Window Short Box Show Truck Black Int. 350 Auto PS PDB Chome
5:1955 Chevrolet Bel Air Ruby Red Metallic 327, Auto PS
6:1955 Chevrolet Bel Air 265 Auto, PS PB Frame-Off Restored
7:1955 Chevrolet Bel Air 265 V8 Auto, Continental Kit
8:1957 Chevrolet Bel Air Frame-off 283 Power Pack Auto
9:1957 Chevrolet Bel Air Frame-Off 283 Auto PS Cont Kit
10:1958 Chevrolet Biscayne 327/425hp  4-speed
11:1971 Chevrolet Blazer 4WD Extensive Restoration
12:1972 Chevrolet Blazer 4WD 350 4-Spd PS PB AC Plaid
13:1967 Chevrolet Camaro Electric Blue 5.7L 700R4 AC PS PB
14:1967 Chevrolet Camaro #s Matching 327 Auto PS PB AC
15:1967 Chevrolet Camaro Pro-Tour Frame-Off AC Fo

## Try ScrapedDeal class created from above functions

In [2]:
scraped = ScrapedDeal.fetch()

100%|██████████| 90/90 [00:00<00:00, 2154.49it/s]


In [3]:
scraped[0]

<1967 Chevrolet Camaro Electric Blue 5.7L 700R4 AC PS PB>

## Test messaging agent

In [3]:
from agents.messaging_agent import MessagingAgent

agent = MessagingAgent()

agent.push('BIG NEWS!!!')

## Test planning agent

In [1]:
import chromadb
DB = 'vectorstore'
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection('cars')


In [2]:
# automatically reload changed modules
%load_ext autoreload
%autoreload 2

from agents.planning_agent import PlanningAgent


In [3]:
planner = PlanningAgent(collection)
planner.plan()

100%|██████████| 90/90 [00:00<00:00, 3545.51it/s]


[Opportunity(deal=Deal(product_description='One of one. Award winning, rotisserie restored, Light Blue Metallic 1970 Dodge Charger R/T with very rare color combination EB3 paint and two tone B5 and B7 blue bucket seat interior and black vinyl top. \xa0According to the Chrysler Registry Report, this rare Charger RT, is only one of one produced, with its rare options and color combination. \xa0This incredible Mopar muscle car has benefited from an extensive rotisserie restoration, before going on to win both the Boca Ratan Concours d\'elegance in 2017 for Best Muscle Car and the Lake Mirror Concours d\'Elegance for Best Muscle car in 2018. This 1970 Charger RT is heavily optioned with a 440 c.i. Magnum engine, Pistol Grip 4-speed transmission, power brakes and A33 Track Pac Dana rear axle with 3.54 gears. This investment grade R/T Charger has had a no expense spared restoration, including the undercarriage and trunk, making it a potential show winner wherever it shows. This American Drea

In [7]:
planner